In [2]:
import libcst


# string_quote_literal_style = "'''"
string_quote_literal_style = '"""'


def has_docstring(
  node: libcst.FunctionDef,
) -> bool:
  if node.body.body and isinstance(node.body.body[0], libcst.SimpleStatementLine) and node.body.body[0].body and isinstance(node.body.body[0].body[0], libcst.Expr) and isinstance(node.body.body[0].body[0].value, libcst.SimpleString):
    return True
  return False


def create_docstring_node(docstring_value: str) -> libcst.SimpleStatementLine:
  docstring = libcst.SimpleStatementLine(body=[
    libcst.Expr(value=libcst.SimpleString(value=f'{string_quote_literal_style}{docstring_value}{string_quote_literal_style}'))
  ])

  return docstring


def insert_docstring_node(updated_node: libcst.FunctionDef, docstring: libcst.SimpleStatementLine) -> libcst.FunctionDef:
  new_body = libcst.IndentedBlock(body=[docstring] + list(updated_node.body.body))
  
  return updated_node.with_changes(body=new_body)


class InsertDocstringTransformer(libcst.CSTTransformer):
  def leave_FunctionDef(self, original_node: libcst.FunctionDef, updated_node: libcst.FunctionDef) -> libcst.FunctionDef:
    # Skip if function already has docstring
    if has_docstring(updated_node):
      return updated_node
    
    # Create docstring node
    docstring = create_docstring_node("Placeholder docstring.")
    
    # Insert docstring as first statement in node
    updated_node = insert_docstring_node(updated_node, docstring)
    
    # Return updated node
    return updated_node


# Code to transform
source_code_path = "example_source.py"
with open(source_code_path) as file:
  code = file.read()

# Parse code
module = libcst.parse_module(code)

# Apply transformer
transformer = InsertDocstringTransformer()
modified_module = module.visit(transformer)

# Print transformed code
with open(f"modified_{source_code_path}", "w") as file:
  file.write(modified_module.code)